## Import Pool-Model Package

In [1]:
from pool_model_pyo3 import *

## Define Simulation Setings
These settings are predefined by our current simulation.
They directly control properties of the cells.
In this example, we focus on a limited subset of parameters which are relevant for our simulation.

In [2]:
simulation_settings = SimulationSettings()
print(simulation_settings)

SimulationSettings {
    voxel_food_diffusion_constant: 25.0,
    voxel_food_initial_concentration: 12.0,
    domain_size: 3000.0,
    starting_domain_x_low: 1350.0,
    starting_domain_x_high: 1650.0,
    starting_domain_y_low: 1350.0,
    starting_domain_y_high: 1650.0,
    n_bacteria_initial: 400,
    bacteria_mechanics_velocity_reduction: 2.0,
    bacteria_mechanics_radius: 6.0,
    bacteria_interaction_potential_strength: 1.0,
    bacteria_interaction_relative_range: 0.5,
    bacteria_cycle_division_age_max: 70.0,
    bacteria_cycle_growth_rate: 0.1,
    bacteria_cycle_food_threshold: 2.0,
    bacteria_cycle_food_growth_rate_multiplier: 10.0,
    bacteria_cycle_food_division_threshold: 0.8,
    bacteria_food_initial_concentration: 1.0,
    bacteria_food_turnover_rate: 0.025,
    bacteria_food_uptake_rate: 0.05,
    intracellular_concentrations: [
        1.0,
    ],
    turnover_rate: [
        0.025,
    ],
    production_term: [
        0.0,
    ],
    degradation_rate: [
      

In [3]:
output_path = run_simulation(simulation_settings)
# output_path = "out/pool_model/2023-10-31-20:06:34"

Running Simulation


## Read results from json Files
The results of the simulation are saved in json files.
Due to the parallelized nature of the simulation, not all results are in one big json file but rather in multiple batches. We therefore need to combine these batches to obtain a complete set for a given iteration.

In [4]:
import os
from pathlib import Path
import json

def combine_batches(run_directory):
    # Opens all batches in a given directory and stores
    # them in one unified big list
    combined_batch = []
    for batch_file in os.listdir(run_directory):
        f = open(run_directory / batch_file)
        b = json.load(f)["data"]
        combined_batch.extend(b)
    return combined_batch

def get_cells_at_iterations(output_path):
    # Uses the previously defined funtion [combine_batches]
    # to read all stored cells at all iterations and stores
    # them in a dictionary.
    dir = Path(output_path) / "cell_storage/json/"
    runs = [(x, dir / x) for x in os.listdir(dir)]
    result = []
    for (n_run, run_directory) in runs:
        result.extend([{"iteration":int(n_run)} | c for c in combine_batches(run_directory)])
    return result

cells_at_iter = get_cells_at_iterations(output_path)

We want to inspect which entries our generated dataset has. Therefore, we normalize the dict, transforming it into a dataframe.
Afterwards, we display all columns.

In [5]:
import pandas as pd

df = pd.json_normalize(cells_at_iter)
for col in df.columns:
    print(col)

iteration
identifier
element.id
element.parent_id
element.cell.mechanics.pos
element.cell.mechanics.vel
element.cell.mechanics.dampening_constant
element.cell.mechanics.mass
element.cell.interaction.potential_strength
element.cell.interaction.relative_interaction_range
element.cell.interaction.cell_radius
element.cell.cycle.age
element.cell.cycle.division_age
element.cell.cycle.maximum_cell_radius
element.cell.cycle.growth_rate
element.cell.cycle.food_threshold
element.cell.cycle.food_growth_rate_multiplier
element.cell.cycle.food_division_threshold
element.cell.cellular_reactions.intracellular_concentrations
element.cell.cellular_reactions.turnover_rate
element.cell.cellular_reactions.production_term
element.cell.cellular_reactions.degradation_rate
element.cell.cellular_reactions.secretion_rate
element.cell.cellular_reactions.uptake_rate


In [6]:
df

,iteration,identifier,element.id,element.parent_id,element.cell.mechanics.pos,element.cell.mechanics.vel,element.cell.mechanics.dampening_constant,element.cell.mechanics.mass,element.cell.interaction.potential_strength,element.cell.interaction.relative_interaction_range,...,element.cell.cycle.growth_rate,element.cell.cycle.food_threshold,element.cell.cycle.food_growth_rate_multiplier,element.cell.cycle.food_division_threshold,element.cell.cellular_reactions.intracellular_concentrations,element.cell.cellular_reactions.turnover_rate,element.cell.cellular_reactions.production_term,element.cell.cellular_reactions.degradation_rate,element.cell.cellular_reactions.secretion_rate,element.cell.cellular_reactions.uptake_rate
0,14000,"[4, 0]","[4, 0]",None,"[1613.4109817022597, 1516.1428541791042]","[-1.8195459485249187e-121, 2.866537441804487e-...",2.0,1.0,1.0,0.5,...,0.1,2.0,10.0,0.8,[0.0312650047319538],[0.025],[0.0],[0.0],[0.0],[0.05]
1,14000,"[4, 1]","[4, 1]",None,"[1607.2023186993758, 1416.6259199507856]","[0.0, 0.0]",2.0,1.0,1.0,0.5,...,0.1,2.0,10.0,0.8,[0.0312650047319538],[0.025],[0.0],[0.0],[0.0],[0.05]
2,14000,"[4, 2]","[4, 2]",None,"[1632.8213479770684, 1355.0900572042501]","[4.464147585475396e-121, 1.692939743883601e-122]",2.0,1.0,1.0,0.5,...,0.1,2.0,10.0,0.8,[0.0312650047319538],[0.025],[0.0],[0.0],[0.0],[0.05]
3,14000,"[4, 3]","[4, 3]",None,"[1418.6803700217733, 1575.3136367478228]","[0.0, 0.0]",2.0,1.0,1.0,0.5,...,0.1,2.0,10.0,0.8,[0.0312650047319538],[0.025],[0.0],[0.0],[0.0],[0.05]
4,14000,"[4, 4]","[4, 4]",None,"[1442.9356418243508, 1591.2510631401663]","[0.0, 0.0]",2.0,1.0,1.0,0.5,...,0.1,2.0,10.0,0.8,[0.0312650047319538],[0.025],[0.0],[0.0],[0.0],[0.05]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32395,7500,"[4, 395]","[4, 395]",None,"[1550.0737894503677, 1578.6128934873675]","[-7.723084935022874e-65, -4.260279168294037e-65]",2.0,1.0,1.0,0.5,...,0.1,2.0,10.0,0.8,[0.15807652835409997],[0.025],[0.0],[0.0],[0.0],[0.05]
32396,7500,"[4, 396]","[4, 396]",None,"[1427.261445454189, 1430.2563177269126]","[0.0, 0.0]",2.0,1.0,1.0,0.5,...,0.1,2.0,10.0,0.8,[0.15807652835409997],[0.025],[0.0],[0.0],[0.0],[0.05]
32397,7500,"[4, 397]","[4, 397]",None,"[1605.9103321222724, 1518.9297484366357]","[-5.798763941898381e-66, 1.881397914819435e-65]",2.0,1.0,1.0,0.5,...,0.1,2.0,10.0,0.8,[0.15807652835409997],[0.025],[0.0],[0.0],[0.0],[0.05]
32398,7500,"[4, 398]","[4, 398]",None,"[1475.7864640871283, 1566.6450023781194]","[-7.995101334972784e-65, -5.943297936632326e-66]",2.0,1.0,1.0,0.5,...,0.1,2.0,10.0,0.8,[0.15807652835409997],[0.025],[0.0],[0.0],[0.0],[0.05]


## Plot Colony
We plot the bacterial colony using `matplotlib`. All particles are represented by 2D spheres and so we can display them as such.

In [7]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import multiprocessing as mp

def save_snapshot(iteration):
    # Filter for only particles at the specified iteration
    df_filtered = df[df["iteration"]==iteration]

    # Get positions as large numpy array
    positions = np.array([np.array(x) for x in df_filtered["element.cell.mechanics.pos"]])
    s = np.array([np.array(x) for x in df_filtered["element.cell.interaction.cell_radius"]])
    c = np.array([np.array(x) for x in df_filtered["element.cell.cellular_reactions.intracellular_concentrations"]])
    norm = matplotlib.colors.Normalize(
        vmin=c.min(),
        vmax=c.max(),
        clip=True,
    )
    mapper = matplotlib.cm.ScalarMappable(norm=norm, cmap=matplotlib.cm.summer)
    c = mapper.to_rgba(c)

    fig, ax = plt.subplots()

    for pos, si, ci in zip(positions, s, c):
        circle = plt.Circle(pos, radius=si, facecolor=ci, edgecolor='k')
        ax.add_patch(circle)

    x_low = positions[:,0].min()
    x_high = positions[:,0].max()
    x_middle = (x_low+x_high)/2
    y_low = positions[:,1].min()
    y_high = positions[:,1].max()
    y_middle = (y_low+y_high)/2.0

    factor = 1.1
    ax.set_xlim([x_middle + factor*(x_low-x_middle), x_middle + factor*(x_high-x_middle)])
    ax.set_ylim([y_middle + factor*(y_low-y_middle), y_middle + factor*(y_high-y_middle)])

    fig.savefig(Path(output_path) / "snapshot_{:08}.png".format(iteration))
    plt.close(fig)

for i in np.unique(df["iteration"]):
    save_snapshot(i)